In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [13]:
# 加载您的数据
data = pd.read_csv(r"D:\ArcGISProjects\workspace\frequency_filter\dy_frequency.csv")  

In [14]:
data.fillna(data.select_dtypes(include='number').mean(),inplace=True)

In [15]:
data.columns

Index(['TL', 'YL', 'TS', 'TZ', 'DEM', 'AnalyticalHillshading', 'Aspect',
       'ChannelNetworkBaseLevel', 'ChannelNetworkDistance',
       'ClosedDepressions', 'ConvergenceIndex', 'LSFactor', 'PlanCurvature',
       'ProfileCurvature', 'RelativeSlopePosition', 'Slope',
       'TopographicWetnessIndex', 'TotalCatchmentArea', 'ValleyDepth',
       'Contrast', 'Correlation', 'Dissimilarity', 'Entropy', 'Homogeneity',
       'Mean', 'ndvi', 'PCA_0', 'PCA_1', 'SecondMoment', 'Variance', 'PRE',
       'SRA', 'TMP', 'VAP', 'WIND', 'BIO', 'LON', 'LAT'],
      dtype='object')

In [23]:
feature_columns = ['DEM', 'AnalyticalHillshading', 'Aspect',
       'ChannelNetworkBaseLevel', 'ChannelNetworkDistance',
       'ClosedDepressions', 'ConvergenceIndex', 'LSFactor', 'PlanCurvature',
       'ProfileCurvature', 'RelativeSlopePosition', 'Slope',
       'TopographicWetnessIndex', 'TotalCatchmentArea', 'ValleyDepth',
       'Contrast', 'Correlation', 'Dissimilarity', 'Entropy', 'Homogeneity',
       'Mean', 'ndvi', 'PCA_0', 'PCA_1', 'SecondMoment', 'Variance', 'PRE',
       'SRA', 'TMP', 'VAP', 'WIND', 'BIO', 'LON', 'LAT']

In [24]:
len(feature_columns)

34

In [35]:
# 选择您的环境因子列和土壤类型列
columns = feature_columns  # 替换为您的环境因子列名
soil_type_column = 'TL'  # 替换为您的土壤类型列名

# 确定要在多少特征的峰值内保留样本
peak_features_required = 7  # 例如，需要在至少2个特征的峰值区域内

# 分别对每个土壤类型进行处理
unique_soil_types = data[soil_type_column].unique()
representative_samples = pd.DataFrame()

for soil_type in unique_soil_types:
    soil_data = data[data[soil_type_column] == soil_type]
    peak_regions = {}

    # 计算每个环境因子的峰值区域
    for col in columns:
        counts, bin_edges = np.histogram(soil_data[col], bins='auto')
        peak_index = np.argmax(counts)
        peak_region = (bin_edges[peak_index], bin_edges[peak_index + 1])
        peak_regions[col] = peak_region

    # 筛选出同时在指定数量的特征峰值区域内的样本
    for index, row in soil_data.iterrows():
        peak_count = sum(row[col] >= low and row[col] <= high for col, (low, high) in peak_regions.items())
        if peak_count >= peak_features_required:
            representative_samples = pd.concat([representative_samples, pd.DataFrame([row])], ignore_index=True)

# 删除重复样本
representative_samples = representative_samples.drop_duplicates()



In [36]:
# 输出结果
print(representative_samples['TL'].value_counts())

黄壤         1243
水稻土        1001
None        755
石灰土         603
紫色土          26
山地灌丛草甸土      21
Name: TL, dtype: int64


In [39]:
len(representative_samples)

3649

In [38]:
representative_samples.to_csv(r"C:\Users\Runker\Desktop\frequency.csv")